In [ ]:
import logging
from pathlib import Path

import matplotlib.pyplot as plt
from darts_postprocessing.prepare_export import prepare_export
from darts_preprocessing.preprocess import load_and_preprocess_planet_scene
from darts_segmentation.segment import SMPSegmenter
from rich import traceback
from rich.logging import RichHandler

from darts.utils.earthengine import init_ee
from darts.utils.logging import setup_logging

setup_logging()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)
traceback.install(show_locals=False)
init_ee("ee-tobias-hoelzer")

In [2]:
DATA_ROOT = Path("../data")

# fpath = DATA_ROOT / "input/planet/PSOrthoTile/4372514/5790392_4372514_2022-07-16_2459"
fpath = DATA_ROOT / "input/planet/PSOrthoTile/4974017/5854937_4974017_2022-08-14_2475"
arcticdem_dir = DATA_ROOT / "input/ArcticDEM"
cache_dir = DATA_ROOT / "download"

In [ ]:
tile = load_and_preprocess_planet_scene(fpath, arcticdem_dir, cache_dir)
tile

In [ ]:
tile_low_res = tile.coarsen(x=16, y=16, boundary="trim").mean()
fig, axs = plt.subplots(2, 6, figsize=(30, 10))
axs = axs.flatten()
for i, v in enumerate(tile_low_res.data_vars):
    tile_low_res[v].plot(ax=axs[i], cmap="gray")
    axs[i].set_title(v)

In [ ]:
model = SMPSegmenter("../models/RTS_v6_tcvis.pt")
tile = model.segment_tile(tile, batch_size=4)
tile

In [ ]:
final_low_res = tile.coarsen(x=16, y=16, boundary="trim").mean()
fig, axs = plt.subplots(2, 7, figsize=(36, 10))
axs = axs.flatten()
for i, v in enumerate(final_low_res.data_vars):
    if v == "probabilities":
        final_low_res[v].plot(ax=axs[i], cmap="gray", vmin=0, vmax=1)
    else:
        final_low_res[v].plot(ax=axs[i], cmap="gray")
    axs[i].set_title(v)

In [ ]:
final = prepare_export(tile)
final

In [ ]:
final_low_res = final.coarsen(x=16, y=16, boundary="trim").mean()
fig, axs = plt.subplots(2, 7, figsize=(36, 10))
axs = axs.flatten()
for i, v in enumerate(final_low_res.data_vars):
    if v == "probabilities":
        final_low_res[v].plot(ax=axs[i], cmap="gray", vmin=0, vmax=100)
    else:
        final_low_res[v].plot(ax=axs[i], cmap="gray")
    axs[i].set_title(v)